In [ ]:
# general imports
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import time
import numpy as np

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

Turning quantum nodes into Keras Layers
=======================================

.. meta::
    :property="og:description": Learn how to create hybrid ML models in PennyLane using Keras
    :property="og:image": https://pennylane.ai/qml/_static/demonstration_assets//Keras_logo.png

.. related::

   tutorial_qnn_module_torch Turning quantum nodes into Torch Layers

*Author: Tom Bromley — Posted: 02 November 2020. Last updated: 28 January 2021.*

Creating neural networks in `Keras <https://keras.io/>`__ is easy. Models are constructed from
elementary *layers* and can be trained using a high-level API. For example, the following code
defines a two-layer network that could be used for binary classification:

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install scikit-learn

In [ ]:
import tensorflow as tf

tf.keras.backend.set_floatx('float64')

layer_1 = tf.keras.layers.Dense(2)
layer_2 = tf.keras.layers.Dense(2, activation="softmax")

model = tf.keras.Sequential([layer_1, layer_2])
model.compile(loss="mae")

# The model can then be trained using `model.fit()
# <https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit>`__.
#
# **What if we want to add a quantum layer to our model?** This is possible in PennyLane:
# :doc:`QNodes <../glossary/hybrid_computation>` can be converted into Keras layers and combined
# with the wide range of built-in classical
# `layers <https://www.tensorflow.org/api_docs/python/tf/keras/layers>`__ to create truly hybrid
# models. This tutorial will guide you through a simple example to show you how it's done!
#
# .. note::
#
#     A similar demo explaining how to
#     :doc:`turn quantum nodes into Torch layers <tutorial_qnn_module_torch>`
#     is also available.
#
# Fixing the dataset and problem
# ------------------------------
#
# Let us begin by choosing a simple dataset and problem to allow us to focus on how the hybrid
# model is constructed. Our objective is to classify points generated from scikit-learn's
# binary-class
# `make_moons() <https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html>`__ dataset:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

X, y = make_moons(n_samples=200, noise=0.1)
y_hot = tf.keras.utils.to_categorical(y, num_classes=2)  # one-hot encoded labels

c = ["#1f77b4" if y_ == 0 else "#ff7f0e" for y_ in y]  # colours for each class
plt.axis("off")
plt.scatter(X[:, 0], X[:, 1], c=c)
plt.show()

# Defining a QNode
# ----------------
#
# Our next step is to define the QNode that we want to interface with Keras. Any combination of
# device, operations and measurements that is valid in PennyLane can be used to compose the
# QNode. However, the QNode arguments must satisfy additional :doc:`conditions
# <code/api/pennylane.qnn.KerasLayer>` including having an argument called ``inputs``. All other
# arguments must be arrays or tensors and are treated as trainable weights in the model. We fix a
# two-qubit QNode using the
# :doc:`default.qubit <code/api/pennylane.devices.default_qubit.DefaultQubit>` simulator and
# operations from the :doc:`templates <introduction/templates>` module.

In [ ]:
# Quantum simulator/QPU
#dev = qml.device("lightning.qubit", wires=n_qubits)
#dev = qml.device("default.qubit", wires=4)
#n_shots=1000
n_shots=2000
n_qubits=2
#dev = qml.device("lightning.qubit", wires=tot_qubits)
dev = qml.device('braket.aws.qubit', device_arn='arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy', wires=n_qubits, shots=n_shots)
#dev = qml.device('braket.aws.qubit', device_arn='arn:aws:braket:::device/quantum-simulator/amazon/sv1', wires=n_qubits, shots=n_shots)

In [ ]:
import pennylane as qml

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}

In [ ]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

In [ ]:
clayer_1 = tf.keras.layers.Dense(2)
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")
model = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss="mae", metrics=["accuracy"])

In [ ]:
fitting = model.fit(X, y_hot, epochs=6, batch_size=5, validation_split=0.25, verbose=2)

In [ ]:
clayer_1 = tf.keras.layers.Dense(4)
qlayer_1 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
qlayer_2 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")

# construct the model
inputs = tf.keras.Input(shape=(2,))
x = clayer_1(inputs)
x_1, x_2 = tf.split(x, 2, axis=1)
x_1 = qlayer_1(x_1)
x_2 = qlayer_2(x_2)
x = tf.concat([x_1, x_2], axis=1)
outputs = clayer_2(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss="mae", metrics=["accuracy"])

fitting = model.fit(X, y_hot, epochs=6, batch_size=5, validation_split=0.25, verbose=2)